# Introduction

We explore the forest surfaces evolution in Romania between 1990 and 2019.

# Data preparation

## Load packages

In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime as dt
%matplotlib inline
import datetime as dt
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from shapely.geometry import shape, Point, Polygon
import folium
from folium.plugins import HeatMap, HeatMapWithTime
init_notebook_mode(connected=True)

## Load the data

In [2]:
data_df = pd.read_csv("/kaggle/input/forest-surfaces-evolution-in-romania-19902019/forest_surfaces_romania_en_1990_2019.csv")

We check the data shape, and we glimpse the data.

In [3]:
data_df.shape

(8111, 5)

In [4]:
data_df.head()

,Category,Region,Year,Unit,Value
0,Total,TOTAL,1990,Thousands hectares,6371.0
1,Total,TOTAL,1991,Thousands hectares,6367.6
2,Total,TOTAL,1992,Thousands hectares,6368.2
3,Total,TOTAL,1993,Thousands hectares,6366.9
4,Total,TOTAL,1994,Thousands hectares,6369.0


In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8111 entries, 0 to 8110
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  8111 non-null   object 
 1   Region    8111 non-null   object 
 2   Year      8111 non-null   int64  
 3   Unit      8111 non-null   object 
 4   Value     8111 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 317.0+ KB


We look to the values of `Category` and of `Region`.

In [6]:
data_df.Category.unique()

array(['Total', 'Wood', 'Softwood', 'Hardwood', 'Others'], dtype=object)

In [7]:
print(list(data_df.Region.unique()))

['TOTAL', 'MACROREGIUNEA UNU', 'Regiunea NORD-VEST', 'Bihor', 'Bistrita-Nasaud', 'Cluj', 'Maramures', 'Satu Mare', 'Salaj', 'Regiunea CENTRU', 'Alba', 'Brasov', 'Covasna', 'Harghita', 'Mures', 'Sibiu', 'MACROREGIUNEA DOI', 'Regiunea NORD-EST', 'Bacau', 'Botosani', 'Iasi', 'Neamt', 'Suceava', 'Vaslui', 'Regiunea SUD-EST', 'Braila', 'Buzau', 'Constanta', 'Galati', 'Tulcea', 'Vrancea', 'MACROREGIUNEA TREI', 'Regiunea SUD-MUNTENIA', 'Arges', 'Calarasi', 'Dambovita', 'Giurgiu', 'Ialomita', 'Prahova', 'Teleorman', 'Regiunea BUCURESTI - ILFOV', 'Ilfov', 'Bucuresti', 'MACROREGIUNEA PATRU', 'Regiunea SUD-VEST OLTENIA', 'Dolj', 'Gorj', 'Mehedinti', 'Olt', 'Valcea', 'Regiunea VEST', 'Arad', 'Caras-Severin', 'Hunedoara', 'Timis']


# Data exploration


Let's plot the total (i.e for all Romania) forest surface evolution from 1990 to 2019 (as total surface, surface with Hardwood, surface with Softwood and Other forest surfaces).

In [8]:
total = data_df.loc[data_df.Region=='TOTAL']
totalT = total.loc[total.Category=='Total']
totalH = total.loc[total.Category=='Hardwood']
totalS = total.loc[total.Category=='Softwood']
totalO = total.loc[total.Category=='Others']

traceT = go.Scatter(
    x = totalT['Year'],y = totalT['Value'],
    name="Total surface",
    marker=dict(color="Green"),
    mode = "markers+lines",
    text=totalT['Value']
)

traceH = go.Scatter(
    x = totalH['Year'],y = totalH['Value'],
    name="Hardwood",
    marker=dict(color="Lightgreen"),
    mode = "markers+lines",
    text=totalH['Value']
)

traceS = go.Scatter(
    x = totalS['Year'],y = totalS['Value'],
    name="Softwood",
    marker=dict(color="Darkgreen"),
    mode = "markers+lines",
    text=totalS['Value']
)

traceO = go.Scatter(
    x = totalO['Year'],y = totalO['Value'],
    name="Other surfaces",
    marker=dict(color="Brown"),
    mode = "markers+lines",
    text=totalO['Value']
)

data = [traceT, traceH, traceS, traceO]

layout = dict(title = 'Total surfaces, Hardwood, Softwood and other surfaces - forest Romania 1990-2019',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Surface [Thousands hectares]'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='forest-surface')

We can observe that overall, the forest surface increased in Romania from 1990 to 2019. Mostly the Hardwood surface increased, from 4.32 Million hectares to 4.51 Million hectares. The total surface increased from 6.37 Million hectares to 6.59 Million hectares. The Softwood surface decreased from 1.92 Million hectares to 1.91 Million hectares.


Let's explore now the same values split per regions and per county.

In [9]:
county_list = ['Bihor', 'Bistrita-Nasaud', 'Cluj', 'Maramures', 'Satu Mare', 'Salaj', 'Alba', 'Brasov', 'Covasna', 'Harghita', 'Mures', 'Sibiu', 
               'Bacau', 'Botosani', 'Iasi', 'Neamt', 'Suceava', 'Vaslui', 'Braila', 'Buzau', 'Constanta', 'Galati', 'Tulcea', 'Vrancea', 
               'Arges', 'Calarasi', 'Dambovita', 'Giurgiu', 'Ialomita', 'Prahova', 'Teleorman', 'Ilfov', 'Bucuresti', 
               'Dolj', 'Gorj', 'Mehedinti', 'Olt', 'Valcea', 'Arad', 'Caras-Severin', 'Hunedoara', 'Timis']
data_county_df = data_df.loc[data_df.Region.isin(county_list)]

In [10]:
selection_df = data_county_df.loc[data_county_df.Category=="Total"]
min_value = selection_df.Value.min()
max_value = selection_df.Value.max()
import plotly.express as px
fig = px.bar(selection_df, x="Value", y="Region", animation_frame='Year', orientation='h',
             range_color =[min_value,max_value],
             width=600, height=800, range_x = [min_value,max_value],
            title='Surface evolution/county 1990-2019 - Total')
fig.update_layout(font=dict(family="Courier New, monospace",size=10,color="#7f7f7f"))
fig.show()

In [11]:
selection_df = data_county_df.loc[data_county_df.Category=="Hardwood"]
min_value = selection_df.Value.min()
max_value = selection_df.Value.max()
import plotly.express as px
fig = px.bar(selection_df, x="Value", y="Region", animation_frame='Year', orientation='h',
             range_color =[min_value,max_value],
             width=600, height=800, range_x = [min_value,max_value],
            title='Surface evolution/county 1990-2019 - Hardwood')
fig.update_layout(font=dict(family="Courier New, monospace",size=10,color="#7f7f7f"))
fig.show()

In [12]:
selection_df = data_county_df.loc[data_county_df.Category=="Softwood"]
min_value = selection_df.Value.min()
max_value = selection_df.Value.max()
import plotly.express as px
fig = px.bar(selection_df, x="Value", y="Region", animation_frame='Year', orientation='h',
             range_color =[min_value,max_value],
             width=600, height=800, range_x = [min_value,max_value],
            title='Surface evolution/county 1990-2019 - Softwood')
fig.update_layout(font=dict(family="Courier New, monospace",size=10,color="#7f7f7f"))
fig.show()

In [13]:
selection_df = data_county_df.loc[data_county_df.Category=="Others"]
min_value = selection_df.Value.min()
max_value = selection_df.Value.max()
import plotly.express as px
fig = px.bar(selection_df, x="Value", y="Region", animation_frame='Year', orientation='h',
             range_color =[min_value,max_value],
             width=600, height=800, range_x = [min_value,max_value],
            title='Surface evolution/county 1990-2019 - Other surfaces')
fig.update_layout(font=dict(family="Courier New, monospace",size=10,color="#7f7f7f"))
fig.show()